# Limit-Cycle Detection via Trajectory Optimization

## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

In [ ]:
# others
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# drake
from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq)

from underactuated import FindResource

## Problem Description

In this exercise we will write a nonlinear optimization to find the limit cycle of the passive compass gait.
We have already discussed this problem in the [lecture notes](http://underactuated.mit.edu/simple_legs.html#compass_gait) under some simplifying assumptions (the stance foot acts as a pin joint does not slip, we disregarded firction limits in the heel strike).
In the case of the rimless wheel, we have also seen how to use `DirectCollocation` to [detect the limit cycle under these simplifying assumptions](https://github.com/RussTedrake/underactuated/blob/master/underactuated/rimless_wheel/dircol_limitcycle.py).
In this notebook we take a more general approach, that can be used as a starting point to derive limit cycles for more complex robots, like the [kneed walker](http://underactuated.mit.edu/simple_legs.html#kneed_walker) or even 3D robots.

We start from the `urdf` description of the compass gait: no derivation of the equations of motion by hand here.
We describe the system in floting-base coordinates, meaning that the position of the stance foot is not fixed and is included in the configuration vector $\mathbf{q}$.
We then parse the `urdf` to obtain the manipulator equations for the compass gait, together with the Jacobian matrices needed to take care of contacts and no-penetration constraints.
Finally, we write a `MathematicalProgram` to identify the limit cycle.
This is a different kind of trajectory optimization from the ones you have been seen before: there is no control action to optimize this time, the only knob we have is the initial state of the compass gait.
We do not even have a cost function to minimize.
But, as you will see, this problem is harder than it seems!

Your goal is to complete the mathematical program we partially wrote below.

# The Model

In the figure below we depicted the compass gait with the system of coordinates we will use in this exercise.
The position of the stance foot, with respect to a frame aligned with the ground, has coordinates $x, y$.
The absolute angle of the stance leg is $\theta_1$, and the angle of the swing leg relative to the stance leg is $\theta_2$.
THe configuration vector is $\mathbf{q} = [x, y, \theta_1, \theta_2]^T$.
The system state is $\mathbf{x} = [\mathbf{q}^T, \dot{\mathbf{q}}^T]^T$.
The bodies with mass are the two legs, and the body (white circle).
You youa are curious to know the numeric parameters of this system, have a look at its [urdf file](https://github.com/RussTedrake/underactuated/blob/master/underactuated/models/compass_gait.urdf).

FIGURE HERE

The idea is simple: we need to find an initial state so that, after a walking cycle, the robot comes back exactly to the same state.
Since the robot is completely symmetric (the legs have the same length and mass distribution), there is no need to optimize for the whole walking cycle (two steps).
We just optimize a single step, then we "mirror" the result to obtain the second step and complete the walking cycle (see the figure below).

FIGURE HERE


**MultibodyPlant scuffing**

## The Walking Cycle

We require the robot to start its motion with both the feet on the ground (figure on the left).
This is the only constraint we impose to the initial state $\mathbf{x}(0)$.

The stance foot must be in contact with the ground for all times ($y(t)=0$), while the swing foot is allowed to leave the ground but, of course, not to penetrate it.
The contact force at the stance foot must always lie in the friction cone.

At the end of the step, the swing foot must collide (i.e. be in contact) with the ground: this moment is called the "heel strike" and denoted as $t_c$.
At the heel strike we have an implusive force on the swing foot which must also lie in the friction cone ($\int_{t_c^-}^{t_c^+} \lambda dt$ [in the lecture notes](http://underactuated.csail.mit.edu/multibody.html#impulsive_collision).).
This impulse leads to a jump in the system state as described [also here](http://underactuated.mit.edu/simple_legs.html#compass_gait).

To ensure the periodicity of the motion we can proceed as follows.
The velocity $\dot{\mathbf{q}}(t)$ evolves contiously up to the heel strike, then, as said, it has a jump.
Consider the system state $\mathbf{x}(t_c^+) = [\mathbf{q}^T(t_c), \dot{\mathbf{q}}^T(t_c^+)]^T$ immediately after this velocity jump.
We want to change coordintes instantaneously, and interchange the stance and the swing leg.


, with the swing foot being the new floating base.



We want the swing foot to be


If we were to change system of coordinates instantaneuosly, and move the floating base to the swing foot

the final configuration $\mathbf{q}$ must mirror  the initial one exactly (as in the figure above).
The velocity contraints require some more careful thinking.
The velocity $\dot{\mathbf{q}}$ evolves contiously up to the 

At the end of the step, the swing foot must be again on the ground, and the final configuration of the robot must mirror perfectly the initial one.


but we do not constrain the initial values of the angles $\theta_1$, $\theta_2$

## Parse the `urdf` and Get the `MultibodyPlant`

Here are a couple of physical parameters we will need below.

In [ ]:
# friction coefficient between feet and ground
friction = 1.

# position of the feet in the respective leg frame
# (has to match the urdf)
foot_in_leg = np.array([0, 0, -1])

Here we parse the `urdf` file.

**An important implementation detail.**
Behind the scenes, the solvers we use for trajectory optimization evaluate at each iteration the constraints and the cost function.
Additionally, they also need to compute the derivaties of these vaues with respect to the optimization variables.
This is needed to understand in which direction the current solution should be perturbed to find a feasible point or reduce the cost.
This process used to be vey tedious some years ago: these solvers required you to write down the deivatives of the cost and the constraints "by hand".
Nowadays, we use [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation), which through the construction of a computational graph is able to evaluate a functions and its derivatives exactly, very quickly.
To allow the evaluation of the `MultibodyPlant` functions (e.g. the computation of the mass matrix) with `AutoDiffXd` variables, we need to call the `MultibodyPlant.ToAutoDiffXd()` function which returns a copy of the `MultibodyPlant` that can work with autodiff variables.

In [ ]:
# parse urdf and create multibody plant
plant = MultibodyPlant(time_step=0)
file_name = FindResource('models/compass_gait_abs.urdf')
Parser(plant).AddModelFromFile(file_name)
plant.Finalize()

# overwrite plant with its autodiff copy
plant = plant.ToAutoDiffXd()

# number of configuration variables
nq = plant.num_positions()

# number of components of the contact forces
nf = 2

## Helper Functions for the `MathematicalProgram`

When writing a `MathematicalProgram` in Drake, optimization variables are `symbolic.Variable` objects.
These cannot be passed directly to the `MultibodyPlant` functions (such as `CalcMassMatrix`), which only accepts floats or `AutoDiffXd` types.
Hence, if you need to add a constraint which involves the evaluation of a `MultibodyPlant` function, you need to proceed as follows.

Write a python function (`my_fun`) which takes some optimization variables (`vars`) as inputs, and returns the quantity (`value`) that you want to constrain.
Then using the method `AddConstraint` from `MathematicalProgram` you can write    `prog.AddConstraint(my_fun, lb=value_lb, ub=value_ub, vars=vars)` to enforce the constraints `value_lb <= value <= value_ub`, where `value_lb` and `value_ub` are vectors of floats of appropriate dimensions.

In the following cell we wrote the functions that we will need to enforce the necessary constraints in the trajectory optimization problem.

In [ ]:
# Function that given the current configuration, velocity,
# acceleration, and contact force at the stance foot, evaluates
# the manipulator equations. The output of this function is a
# vector with dimensions equal to the number of configuration
# variables. If the output of this function is equal to zero
# then the given inputs verify the manipulator equations.
def manipulator_equations(vars):
    
    # split input vector in subvariables
    # configuration, velocity, acceleration, stance-foot force
    assert vars.size == 3 * nq + nf
    split_at = [nq, 2 * nq, 3 * nq]
    q, qd, qdd, f = np.split(vars, split_at)
    
    # set plant state
    context = plant.CreateDefaultContext()
    plant.SetPositions(context, q)
    plant.SetVelocities(context, qd)
    
    # matrices for the manipulator equations
    M = plant.CalcMassMatrixViaInverseDynamics(context)
    Cv = plant.CalcBiasTerm(context)
    tauG = plant.CalcGravityGeneralizedForces(context)
    
    # Jacobian of the stance foot
    J = get_foot_jacobian(plant, context, 'stance_leg')
    
    # return manipulator equations violation
    return M.dot(qdd) + Cv - tauG - J.T.dot(f)

# This function implements the implusive collision derived in
# the appendix of the book. It take as inputs the compass gait
# configuration, velocity before and after the impact, and the
# time integral of the impulse (in latex: $\int_{t_c^-}^{t_c^+} \lambda dt$).
# It returns a vector of quantities that must vanish in order
# for the impusive dynamics to be verified. More specifically,
# it enforces the velocity jump due to the impulse, and the
# inelastic behavior of the contact (the coefficient of
# restitution $e$ is zero).
# See http://underactuated.mit.edu/multibody.html#impulsive_collision
def reset_velocity_heelstrike(vars):
    
    # split input vector in subvariables
    # qd_pre: generalized velocity before the impact
    # qd_post: generalized velocity after the impact
    # lam_int: time integral of the collision impulse (2d vector)
    assert vars.size == 3 * nq + nf
    split_at = [nq, 2 * nq, 3 * nq]
    q, qd_pre, qd_post, lam_int = np.split(vars, split_at)

    # set plant configuration
    context = plant.CreateDefaultContext()
    plant.SetPositions(context, q)
    
    # get necessary matrices
    M = plant.CalcMassMatrixViaInverseDynamics(context)
    J = get_foot_jacobian(plant, context, 'swing_leg')
    
    # return a vector of quantities that must vanish for the
    # impulsive dynamics to hold
    return np.concatenate((
        M.dot(qd_post - qd_pre) - J.T.dot(lam_int), # velocity jump due to the impulse
        J.dot(qd_post) # inelastic contact, zero velocity restitution
    ))

def foot_position(leg, q):
    
    # get reference frames for the leg and the ground
    leg_frame = plant.GetBodyByName(leg).body_frame()
    ground_frame = plant.GetBodyByName('ground').body_frame()
    
    # position of the foot in ground coordinates
    context = plant.CreateDefaultContext()
    plant.SetPositions(context, q)
    foot_position = plant.CalcPointsPositions(
        context,
        leg_frame,
        foot_in_leg,
        ground_frame 
    )
    
    return foot_position[[0, 2]]

# helper function that given a leg, returns the Jacobian
# matrix for the related foot
def get_foot_jacobian(plant, context, leg):
    
    # get reference frames for the given leg and the ground
    leg_frame = plant.GetBodyByName(leg).body_frame()
    ground_frame = plant.GetBodyByName('ground').body_frame()

    # compute Jacobian
    J = plant.CalcJacobianTranslationalVelocity(
        context,
        JacobianWrtVariable(0), # do not care too much about this
        leg_frame,
        foot_in_leg,
        ground_frame,
        ground_frame
    )
    
    # throw away the y coordinate since we are in 2D
    return J[[0, 2]]

## The Trajectory Optimization Problem

We start by setting some parameters of our optimization problem.

In [ ]:
# numeric parameters
T = 50       # time steps in the trajectory optimization
h_min = .005 # minimum time interval is seconds
h_max = .05  # maximum time interval is seconds

**Troubleshooting.**
To simplify the reading, we divide the construction of the `MathematicalProgram` in multiple cells.
If you modify any of the components of the problem, be sure to rerun your code starting from the following cell where the `MathematicalProgram` is initialized.
Otherwise you will add the same constraints multiple times to the same optimization problem.

We start from the decision variables of the trajectory optimization problem.
Notice that we also add the accelerations `qdd` among the optimization variables here.
This is slightly unusual, and not necessary, but in these circumnstances simplifies a bit the code.

In [ ]:
# initialize program
prog = MathematicalProgram()

# vector of the time intervals
# (distances between the T + 1 break points)
h = prog.NewContinuousVariables(T, name='h')

# system configuration, generalized velocities, and accelerations
q = prog.NewContinuousVariables(rows=T+1, cols=nq, name='q')
qd = prog.NewContinuousVariables(rows=T+1, cols=nq, name='qd')
qdd = prog.NewContinuousVariables(rows=T, cols=nq, name='qdd')

# stance-foot force
f = prog.NewContinuousVariables(rows=T, cols=nf, name='f')

# time integral of the heel strike impulse for the swing leg
lam_int = prog.NewContinuousVariables(nf, name='lam_int')

# generalized velocity after the swing-leg impact with the ground
# after "mirroring", this must coincide with the initial velocity
# qd[0] to ensure periodicity of the motion
qd_post = prog.NewContinuousVariables(nq, name='qd_post')

Here are part of the constraints of the optimization problem.

Now it is your turn to complete the optimization problem.
You will need to add five groups of constraints.
If you do things correctly, you should be able to express all of these as **linear** constraints on the decision variables we defined above.
Hence in the following cell yo should only need to use the method `AddLinearConstraint` of `MathematicalProgram`.
More precisely, you need to:
- Ensure that at time zero the swing foot is in contact with the ground.
Noticing that in the last cell we already ensured that the stance foot is in contact with the ground at time zero, and taking advantage of the simmetry in the structure of the compass gait, you should be able to express this as a linear constraint on the vector `q[0]`.
- Ensure that the contact force at the stance foot lies in the friction cone for all times.
This means that the normal component (`f[t, 1]`) must be nonnegative and the tangential one (`f[t, 0]`) must be between `- f[t, 1]` and `+ f[t, 1]`.
- Ensure that the force at the swing foot during the heel strike lies in the friction cone. (Same as before but for `lam_int`.)
- Require that the final configuration `q[-1]` mirrors exactly the initial one `q[0]`.
- Require that the velocity `qd_post` after the heel strike mirrors exactly the initial velocity `qd[0]`.

Here we set the initial guess for our optimization problem.

For the robot configuration `q`, we just interpolate between the initial value `q0_guess` and the final value `- q0_guess`.
In our implementation, the value given below for `q0_guess` made the optimization converge.
But, if you find the need, feel free to tweak this parameter.
The initial guess for the velocity and the acceleration is obtained by differentiating the one for the position.

For the time step `h` we just initialize it to its maximal value (somewhat an arbitrary decision, feel free to try `h_min` if you feel to), and the normal force at the stance foot is initially equal to the weight of the robot.

All the other optimization variables are initialized at zero.
Note that, if the initial guess for a variable is not specified, the default value is zero.

In [ ]:
# vector of the initial guess
initial_guess = np.empty(prog.num_vars())

# initial guess for the time step
h_guess = h_max
prog.SetDecisionVariableValueInVector(h, [h_guess] * T, initial_guess)

# # interpolate postion
# theta_init = np.pi / 10
# theta_final = - theta_init
# theta_guess = np.linspace(theta_init, theta_final, T + 1)
# q_guess = np.zeros((T + 1, nq))
# for t, theta in enumerate(theta_guess):
#     q_guess[t] = np.array([
#         np.sin(theta_init) - np.sin(theta),
#         np.cos(theta),
#         theta,
#         - theta
#     ])
# q_guess_poly = PiecewisePolynomial.FirstOrderHold(
#     np.arange(0, T + 1) * h_guess,
#     q_guess.T
# )

theta = np.pi / 10
q0_guess = np.array([
    0,
    np.cos(theta),
    theta,
    - theta
])
qf_guess = np.array([
    2 * np.sin(theta),
    np.cos(theta),
    - theta,
    theta
])
q_guess_poly = PiecewisePolynomial.FirstOrderHold(
    [0, T * h_guess],
    np.column_stack((q0_guess, qf_guess))
)
qd_guess_poly = q_guess_poly.derivative()
qdd_guess_poly = q_guess_poly.derivative()
q_guess = np.hstack([q_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
qd_guess = np.hstack([qd_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
qdd_guess = np.hstack([qdd_guess_poly.value(t * h_guess) for t in range(T)]).T

# initial guess for the state
prog.SetDecisionVariableValueInVector(q, q_guess, initial_guess)
prog.SetDecisionVariableValueInVector(qd, qd_guess, initial_guess)
prog.SetDecisionVariableValueInVector(qdd, qdd_guess, initial_guess)

# total weight of the compass gait
bodies = ['body', 'stance_leg', 'swing_leg']
mass = sum(plant.GetBodyByName(body).default_mass() for body in bodies)
g = - plant.gravity_field().gravity_vector()[-1]
weight = mass * g

# initial guess for the normal component of the stance-leg force
prog.SetDecisionVariableValueInVector(f[:, 1], [weight] * T, initial_guess)

# prog.SetDecisionVariableValueInVector(lam_int[1], weight/2, initial_guess)

We can finally solve the problem! Be sure that the solver actually converged; you can do this by looking at the variable `result.is_success()`.

In [ ]:
# solve mathematical program with initial guess
solver = SnoptSolver()
result = solver.Solve(prog, initial_guess)

# ensure solution is found
print('Solution found?', result.is_success())

In the following cell, we retrieve the optimal value of some of the decision variables.

In [ ]:
# get optimal solution
q_opt = result.GetSolution(q)
qd_opt = result.GetSolution(qd)
h_opt = result.GetSolution(h)
x_opt = np.hstack((q_opt, qd_opt))

## Animate the Result

Here we quickly build a Drake diagram to animate the result we got from trajectory optimization: useful for debugging your code and be sure that everything looks good.

In [ ]:
# interpolate state values for animation
time_breaks_opt = np.array([sum(h_opt[:t]) for t in range(T+1)])
x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, x_opt.T)

# parse plant with scene graph
plant = MultibodyPlant(time_step=0)
scene_graph = SceneGraph()
plant.RegisterAsSourceForSceneGraph(scene_graph)
file_name = FindResource('models/compass_gait_abs.urdf')
Parser(plant).AddModelFromFile(file_name)
plant.Finalize()

# build block diagram and drive system state with
# the trajectory from the optimization problem
builder = DiagramBuilder()
source = builder.AddSystem(TrajectorySource(x_opt_poly))
builder.AddSystem(scene_graph)
pos_to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant, input_multibody_state=True))
builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
builder.Connect(pos_to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

# add visualizer
xlim = [-.75, 1.]
ylim = [-.2, 1.5]
visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, xlim=xlim, ylim=ylim, show=False))
builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))
simulator = Simulator(builder.Build())

# generate animation
visualizer.start_recording()
simulator.AdvanceTo(x_opt_poly.end_time())
ani = visualizer.get_recording_as_animation()
display(HTML(ani.to_jshtml()))

## Plot the Results

Here are two plots to visualize the results of the trajectory optimization.

In the first we plot the angles of the two legs.
To complete the plot we mirror the trajctory ...

In [ ]:
plt.plot(q_opt[:, 2], q_opt[:, 3], color='b', label='Blue leg swinging')
plt.plot(q_opt[:, 3], q_opt[:, 2], color='r', label='Red leg swinging')
plt.scatter(q_opt[0, 3], q_opt[0, 2], color='b', zorder=3, label='Blue-leg heel strike')
plt.scatter(q_opt[0, 2], q_opt[0, 3], color='r', zorder=3, label='Red-leg heel strike')
plt.xlabel('Red-leg angle')
plt.ylabel('Blue-leg angle')
plt.grid(True)
plt.legend()

Here we plot angle vs velocity.
If you did things correctly this should look like the one from Figure??? here, upside down.

In [ ]:
plt.plot(q_opt[:, 2], qd_opt[:, 2], color='b', label='Blue leg swinging')
plt.plot(q_opt[:, 3], qd_opt[:, 3], color='r', label='Red leg swinging')
plt.plot(
    [q_opt[0, 2], q_opt[-1, 3]],
    [qd_opt[0, 2], qd_opt[-1, 3]],
    linestyle=':',
    color='r',
    label='Red-leg heel strike'
)
plt.plot(
    [q_opt[-1, 2], q_opt[0, 3]],
    [qd_opt[-1, 2], qd_opt[0, 3]],
    linestyle=':',
    color='b',
    label='Blue-leg heel strike'
)
plt.xlabel('Red-leg angle')
plt.ylabel('Red-leg angular velocity')
plt.grid(True)
plt.legend()